This is my capstone project for the Udacity Machine Learning Nanodegree.

Import the libraries needed.

In [15]:
import pandas as pd
import numpy as np
import keras as kr
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from os import listdir

Get the data.

In [16]:
directory = 'sandp500/individual_stocks_5yr'
dir_listing = listdir(directory)

In [17]:
symbols_list = []

for symbol in dir_listing:
    symb = symbol.split('_')[0]
    symbols_list.append(symb)

print(len(symbols_list))
print(symbols_list[0])

504
AAL


In [18]:
csv_file = '{}/{}_data.csv'.format(directory, symbols_list[0])

Since we already know the name of the specific stock we are trying to get from the name of the file, we can drop that column in the dataframe.

In [19]:
dataset = pd.read_csv(csv_file)

In [20]:
dataset = dataset.assign(trading_date = pd.to_datetime(dataset['Date']))

In [21]:
#dataset = dataset.drop('Name', 1)
dataset = dataset.drop('Date', 1)
dataset.set_index(['trading_date'], inplace=True)

In [22]:
dataset
log_dataset = np.log(dataset)
log_dataset

AttributeError: 'float' object has no attribute 'log'

In [13]:
def getting_preprocessed_data(symbol):
        csv_file = '{}/{}_data.csv'.format(directory, symbol)
        print(csv_file)
        #df = pd.read_csv(csv_file)
        #df = df.drop('Name', 1)
        #df.set_index('Date', inplace=True)
        # below was found at https://stackoverflow.com/questions/29314033/python-pandas-dataframe-remove-empty-cells
        #df['Open'].replace('', np.nan, inplace=True)
        #df.dropna(subset=['Open'], inplace=True)
        #return df

def plotting_stocks(symbols_list, amount_of_stocks=0):
    if amount_of_stocks == 0:
        amount_of_stocks = len(symbols_list)
        
    for symbol in symbols_list[:amount_of_stocks]:
        fig, ax = plt.subplots()
        fig.subplots_adjust(right=0.7)
        df = getting_preprocessed_data(symbol)
        print(symbol)
        df.Open.plot(ax=ax, style='b-', figsize=(20,10))
        # same ax as above since it's automatically added on the right
        df.Volume.plot(ax=ax, style='r-', secondary_y=True, figsize=(20,10))
        # add legend --> take advantage of pandas providing us access
        # to the line associated with the right part of the axis
        #ax.legend([ax.get_lines()[0], ax.get_lines()[0]], ['Open','Volume'], bbox_to_anchor=(1.5, 0.5))
        plt.show()
        #below is the Daily Returns calculation to put into the Sharpe Ratio. 
        df_preprocessed = df.assign(Daily_Returns = np.divide((df.Open - df.Close), df.Close) * 100)

        #Below is the calculation for the Sharpe Ratio column. 
        df_preprocessed = df_preprocessed.assign(Sharpe_Ratio = np.divide((df_preprocessed.Daily_Returns - 0.046), np.std(np.array([df_preprocessed.Open, df_preprocessed.High, df_preprocessed.Low, df_preprocessed.Close]))))

        #Below is the rate of change (momentum) for the specific stock. 
        df_preprocessed = df_preprocessed.assign(Rate_of_Change = (np.divide(df_preprocessed.Close, df_preprocessed.Open) - 1) * 100)

        #df.plot.scatter(x='Open', y='Volume', label="AAL")
        log_df = np.log(df)
        log_df_preprocessed = np.log(df_preprocessed)
        log_df_preprocessed.plot.scatter(x='Volume', y='Open', label="AAL", figsize=(20,10))
        plt.show()
        log_df_preprocessed.plot.scatter(x='Open', y='Sharpe_Ratio', label="Sharpe Ratio Open", figsize=(20,10))
        plt.show()
        log_df_preprocessed.plot.scatter(x='Volume',y='Sharpe_Ratio', label="Sharpe Ratio Volume", figsize=(20,10), use_index=True)
        plt.show()
        log_df_preprocessed.plot.scatter(x='Volume',y='Close', label="Volume Close", figsize=(20,10), use_index=True)
        plt.show()

In [14]:
# printing out the first four stocks to get an idea of how each stock is individually represented.
plotting_stocks(log_dataset, 1)

sandp500/individual_stocks_5yr/Open_data.csv
Open


AttributeError: 'NoneType' object has no attribute 'Open'

In [ ]:
#below is the Daily Returns calculation to put into the Sharpe Ratio. 
dataset_preprocessed = dataset.assign(Daily_Returns = np.divide((dataset.Open - dataset.Close), dataset.Close) * 100)

In [ ]:
#Below is the calculation for the Sharpe Ratio column. 
dataset_preprocessed = dataset_preprocessed.assign(Sharpe_Ratio = np.divide((dataset_preprocessed.Daily_Returns - 0.046), np.std(np.array([dataset_preprocessed.Open, dataset_preprocessed.High, dataset_preprocessed.Low, dataset_preprocessed.Close]), ddof=1)))

In [ ]:
#Below is the rate of change for the specific stock also known as momentum
dataset_preprocessed = dataset_preprocessed.assign(Rate_of_Change = (np.divide(dataset_preprocessed.Close, dataset_preprocessed.Open) - 1) * 100)

In [ ]:
dataset_preprocessed

In [ ]:
test = np.log(np.exp(64))

In [ ]:
print(test)